In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

file_path = 'data/dev_tr_survey_2023.csv'

df = pd.read_csv(file_path)

for column in df.columns:
    if column not in ['Zaman damgası', 'Ağırlıklı olarak hangi dil ve teknolojileri kullanıyorsunuz?', 'Yan haklarınız neler?']:
        column_count = df[column].value_counts()
        column_df = pd.DataFrame({column: column_count.index, 'Count': column_count.values})
        fig = px.pie(column_df, values='Count', names=column)
        fig.update_layout(title_text=column)
        fig.show()
    elif column in ['Ağırlıklı olarak hangi dil ve teknolojileri kullanıyorsunuz?', 'Yan haklarınız neler?']:
        df[column] = df[column].str.replace('doğum günü izni, regl izni gibi', 'doğum günü izni|regl izni gibi')
        df_positions = df[column].str.split(', ').explode()
        position_counts = df_positions.value_counts()
        fig = px.bar(position_counts, x=position_counts.index, y=position_counts.values,
                     labels={'x': column, 'y': 'Count'})
        fig.update_layout(title_text=column,xaxis_tickangle=45)
        fig.show()
    if column == 'Cinsiyet ya da Cinsel Yönelim':
        gender_df = df.groupby(['Cinsiyet ya da Cinsel Yönelim', 'Seviyeniz nedir?']).size().reset_index(name='Count')
        fig1 = px.bar(gender_df, x='Cinsiyet ya da Cinsel Yönelim', y='Count', color='Seviyeniz nedir?',
                      labels={'Cinsiyet ya da Cinsel Yönelim': 'Gender/Sexual Orientation', 'Count': 'Count'})
        fig1.update_layout(title_text='Gender/Sexual Orientation by Experience Level')
        fig1.show()


        fig2 = make_subplots(rows=1, cols=3, subplot_titles=('Junior', 'Mid', 'Senior'), specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]])

        for i, level in enumerate(['Junior', 'Mid', 'Senior']):
            level_df = gender_df[gender_df['Seviyeniz nedir?'] == level]
            pie_fig = px.pie(level_df, values='Count', names='Cinsiyet ya da Cinsel Yönelim')
            fig2.add_trace(pie_fig.data[0], row=1, col=i+1)

        fig2.update_layout(title_text='Gender/Sexual Orientation Distribution by Experience Level')
        fig2.show()

df['Maaş /Aylık Türk Lirası Cinsinden'] = df['Maaş /Aylık Türk Lirası Cinsinden'].str.replace('TL', '', regex=False).str.strip()
df[['Min Salary', 'Max Salary']] = df['Maaş /Aylık Türk Lirası Cinsinden'].str.split('-', expand=True)

df['Max Salary'] = df['Max Salary'].str.replace('ve üzeri', '', regex=False).str.strip()
df.loc[df['Max Salary'] == '250.000', 'Max Salary'] = '250000'
df['Min Salary'] = df['Min Salary'].str.replace('ve üzeri', '', regex=False).str.strip()
df.loc[df['Min Salary'] == '250.000', 'Min Salary'] = '250000'

df['Min Salary'] = df['Min Salary'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Max Salary'] = df['Max Salary'].str.replace('.', '').str.replace(',', '.').astype(float).fillna(10000)


avg_salary_gender = df.groupby('Cinsiyet ya da Cinsel Yönelim')[['Min Salary', 'Max Salary']].mean()
avg_salary_gender['Average Salary'] = avg_salary_gender.mean(axis=1)


avg_salary_gender = avg_salary_gender.reset_index()[['Cinsiyet ya da Cinsel Yönelim', 'Average Salary']]

avg_salary_profession = df.groupby('Hangi pozisyonda çalışıyorsunuz?')[['Min Salary', 'Max Salary']].mean()
avg_salary_profession['Average Salary'] = avg_salary_profession.mean(axis=1)

avg_salary_profession = avg_salary_profession.reset_index()[['Hangi pozisyonda çalışıyorsunuz?', 'Average Salary']]

fig_gender = px.bar(avg_salary_gender, x='Cinsiyet ya da Cinsel Yönelim', y='Average Salary')
fig_gender.update_layout(title='Average Salary by Gender')
fig_gender.show()

avg_salary_profession = avg_salary_profession.sort_values(by='Average Salary', ascending=False)

fig_profession = px.bar(avg_salary_profession, x='Hangi pozisyonda çalışıyorsunuz?', y='Average Salary')
fig_profession.update_layout(title='Average Salary by Profession (Descending Order)',xaxis_tickangle=45)
fig_profession.show()